In [ ]:
import json
import math
import random
import numpy as np

In [ ]:
import random
import time

def str_time_prop(start, end, format):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + random.random() * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end):
    return str_time_prop(start, end, '%d/%m/%Y')

In [ ]:
y1, x1, y2, x2 = 16.8073393, 51.0426686, 17.1762192, 51.2100604
# x, y = x1 + (x2 - x1)/2, y1 + (y2 - y1)/2
# scale_x, scale_y = ((x - x1) + (x2 - x))/2, ((y - y1) + (y2 - y))/2 

size = 10_000
# no_houses = int(size / 100)
no_houses = size
energy_summer = (300, 200)
energy_winter = (-300, 500)

In [ ]:
houses = []
districts = {
    'bieńkowice': ([17.0717411,51.029829,17.1117411,51.069829], 0.5, 10),
    'stare miasto': ([16.8073393,51.0426686,17.1762192,51.2100604], 10.3, 100),
    'krzyki-partynice': ([16.9843425,51.0511937,17.0243425,51.0911937], 19.9, 80),
    'psie pole-zawidawie': ([17.094733,51.1259988,17.134733,51.1659988], 29.6, 60),
    'leśnica': ([16.8458752,51.1321376,16.8858752,51.1721376], 28.1, 70),
    'ołbin': ([17.035126,51.1032183,17.075126,51.1432183], 34.6, 95),
    'pilczyce-kozanów-popowice północne': ([16.9375639,51.1184358,16.9775639,51.1584358], 33.5, 90),
    'kowale': ([17.0817765,51.1120703,17.1217765,51.1520703], 2.3, 20)
}

districts_population = [0.5, 10.3, 19.9, 29.6, 28.1, 34.6, 33.5, 2.3]

for i in range(no_houses):
    district = random.choices(list(districts.keys()), weights=districts_population)[0]
    y1, x1, y2, x2 = districts[district][0]
    x, y = x1 + (x2 - x1)/2, y1 + (y2 - y1)/2
    scale_x, scale_y = ((x - x1) + (x2 - x))/2, ((y - y1) + (y2 - y))/2 
    rand_x, rand_y = np.random.normal(x, scale_x), np.random.normal(y, scale_y)
    rand_traffic = abs(np.random.normal(districts[district][2], 10))
    houses.append((rand_x, rand_y, districts[district][1], rand_traffic))

In [ ]:
points = []
    
for i in range(int(round(size/2))):
    rand_date = random_date("1/04/2019", "30/9/2019")
    rand_x, rand_y, population, traffic = random.choice(houses)
    rand_energy = np.random.normal(energy_summer[0], energy_summer[1])
    points.append({'x': rand_x, 'y': rand_y, 'date': rand_date, 'energy_delta': rand_energy, 'population': population, 'traffic': traffic})
    
for i in range(int(round(size/2))):
    rand_date = random_date("1/01/2019", "31/03/2019") if random.random() < 0.5 else random_date("1/10/2019", "31/12/2019")
    rand_x, rand_y, population, traffic = random.choice(houses)
    rand_energy = np.random.normal(energy_winter[0], energy_winter[1])
    points.append({'x': rand_x, 'y': rand_y, 'date': rand_date, 'energy_delta': rand_energy, 'population': population, 'traffic': traffic})

In [ ]:
points = []
max_energy, min_energy = 0, 0
    
for i in range(size):
    rand_x, rand_y, population, traffic = random.choice(houses)
    rand_energy = np.random.normal(-300, 500) if random.random() < 0.5 else np.random.normal(300, 200)
    if rand_energy < min_energy:
        min_energy = rand_energy
    if rand_energy > max_energy:
        max_energy = rand_energy
    points.append({'x': rand_x, 'y': rand_y, 'energy_delta': rand_energy, 'population': population, 'traffic': traffic})

In [ ]:
with open('data/data_without_date.json', 'w') as points_file:
    json.dump(points, points_file)       

## Charging systems

In [ ]:
charging_systems = [
    [17.023146, 51.174947],
    [17.028005, 51.157343],
    [17.103798, 51.145548],
    [17.127055, 51.155197],
    [17.022672, 51.116371],
    [17.020993, 51.117139],
    [16.988662, 51.118742],
    [16.975578, 51.126633],
    [17.031650, 51.103415],
    [17.027631, 51.101445],
    [17.027833, 51.095360],
    [17.020465, 51.094515],
    [17.059772, 51.081472],
    [17.047815, 51.077390],
    [17.006985, 51.072248],
]

In [ ]:
with open('data/charging_systems.json', 'w') as charging_systems_file:
    json.dump(charging_systems, charging_systems_file)

In [ ]:
y1, x1, y2, x2 = 16.8073393, 51.0426686, 17.1762192, 51.2100604
height, width = y2 - y1, x2 - x1
steps = 10
height_step, width_step = height/steps, width/steps

In [ ]:
def normalize(x, max_x, min_x):
    return (x - min_x)/(max_x - min_x)

In [ ]:
min_dist1, min_dist2 = 10000, 10000
for i in range(steps):
    for j in range(steps):
        x, y = x1 + i * width_step, y2 - i * height_step
        for point in points:
            dist = math.hypot(point['x'] - x, point['y'] - y)
            min_dist1 = dist if dist < min_dist1 else min_dist1
        for system in charging_systems:
            dist = math.hypot(system[1] - x, system[0] - y)
            min_dist2 = dist if dist < min_dist2 else min_dist2

In [ ]:
min_dist1 = 1/(min_dist1*min_dist1)
min_dist2 = 1/(min_dist2*min_dist2)

In [ ]:
possible_charging_systems = []
for i in range(steps):
    for j in range(steps):
        x, y = x1 + i * width_step, y2 - j * height_step
        optim = 0
        for point in points:
            dist = math.hypot(point['x'] - x, point['y'] - y)
            optim += normalize(1/(dist*dist), min_dist1, 0) * \
                normalize(point['energy_delta'], max_energy, min_energy) * \
                normalize(point['population'], 34.6, 0.5) * \
                normalize(point['traffic'], 100, 10)
        for system in charging_systems:
            dist = math.hypot(system[0] - x, system[1] - y)
            optim -= 2 * normalize(1/(dist*dist), min_dist2, 0)
        possible_charging_systems.append((x, y, optim))

In [ ]:
with open('data/possible_charging_system.json', 'w') as possible_charging_systems_file:
    json.dump(possible_charging_systems, possible_charging_systems_file)

In [ ]:
possible_charging_systems